<a href="https://colab.research.google.com/github/thaisneubauer/PPAR2021/blob/main/BPI_Challenge_2012_04PP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicialização do ambiente

## Importando as bibliotecas

In [ ]:
import sys
import math
import random
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

## Instalando biblioteca para tempo de execução

In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 196 µs (started: 2021-07-04 19:03:27 +00:00)


# Carregar os dados

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

google_id = '1DgYTvfbzTe8CPMOwXMupmTOiMTP56PwD'
gdd.download_file_from_google_drive(file_id=google_id, 
                                    dest_path = './data.csv', 
                                    showsize = True)
data = pd.read_csv("data.csv")

30.3 MiB Done.
time: 5.28 s (started: 2021-07-04 19:03:27 +00:00)


In [ ]:
data.head()

,Case ID,Activity,Resource,Workload,Duration(s)
0,173712,W_Fixing incoming lead,10912,"{10912: 'FREE', 10982: 'FREE', 11019: 'FREE', ...",109.0
1,173709,W_Filling in information for the application,10912,"{10912: 'FREE', 10982: 'FREE', 11019: 'FREE', ...",25.0
2,173703,W_Filling in information for the application,10912,"{10912: 'FREE', 10982: 'FREE', 11019: 'FREE', ...",158.0
3,173709,W_Filling in information for the application,10912,"{10912: 'FREE', 10982: 'FREE', 11019: 'FREE', ...",142.0
4,173703,W_Filling in information for the application,10912,"{10912: 'FREE', 10982: 'FREE', 11019: 'HIGH', ...",1306.0


time: 85.2 ms (started: 2021-07-04 19:03:32 +00:00)


In [ ]:
len(data['Case ID'].unique())

6129

time: 11.7 ms (started: 2021-07-04 19:03:32 +00:00)


In [ ]:
len(data['Activity'].unique())

6

time: 16.6 ms (started: 2021-07-04 19:03:32 +00:00)


# Definição do modelo

## Seleção de recursos (ações)

Selecionar 5 recursos aleatoriamente:

In [ ]:
resources = list(data.Resource.unique())
random.seed(42)
random.shuffle(resources)
selected_res = resources[:5]
select_res_data = data[data.Resource.isin(selected_res)]
states = list(select_res_data.Activity.unique())

time: 7.68 ms (started: 2021-07-04 19:03:35 +00:00)


## Definir T 


Probabilidades de transição para cada ação (recurso) em cada estado (atividade):

In [ ]:
ACT_TRANS_PROB = pd.DataFrame(columns=['s','a','s_next','p'])

def get_trans_prob(x):
  total = x['Case ID'].count()
  trans_prob = pd.DataFrame(x['Next Activity'].value_counts()/total).reset_index().rename(columns={'Next Activity':'p', 'index':'s_next'})
  trans_prob['a'] = x['Resource'].iloc[0]
  trans_prob['s'] = x['Activity'].iloc[0]
  global ACT_TRANS_PROB 
  ACT_TRANS_PROB = pd.concat([ACT_TRANS_PROB,trans_prob])
  return x

select_res_data['Next Activity'] = select_res_data['Activity'].shift(-1)
prob_selected_res = (select_res_data[['Resource','Activity', 'Next Activity','Case ID']]
                      .groupby(['Activity','Resource'])
                      .apply(get_trans_prob))

time: 175 ms (started: 2021-07-04 19:03:37 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
T = {}

def fillT(x):
  if x['s'] in T:
    if x['a'] in T[x['s']]:
      T[x['s']][x['a']].append((x['p'], x['s_next']))
    else:
      T[x['s']][x['a']] = [(x['p'], x['s_next'])]
  else:
    T[x['s']] = {x['a']: [(x['p'], x['s_next'])]}

_ = ACT_TRANS_PROB.apply(fillT, axis = 1)

time: 23.2 ms (started: 2021-07-04 19:03:37 +00:00)


Exemplo da estrutura - valores de transição para a primeira atividade:

In [ ]:
atividade1=list(T.keys())[0]
atividade1

'W_Assessing the application'

time: 5.22 ms (started: 2021-07-04 19:03:37 +00:00)


In [ ]:
T[atividade1]
# para a acao (recurso) 10809: 65% de ir para 'W_Assessing the application' e 35% de ir para 'W_Calling after sent offers'

{10863: [(0.5, 'W_Filling in information for the application'),
  (0.25, 'W_Fixing incoming lead'),
  (0.25, 'W_Assessing the application')],
 10909: [(0.5625, 'W_Filling in information for the application'),
  (0.25, 'W_Calling after sent offers'),
  (0.1875, 'W_Calling to add missing information to the application')],
 11179: [(1.0, 'W_Calling to add missing information to the application')]}

time: 6.78 ms (started: 2021-07-04 19:03:37 +00:00)


## Definir R

Definir as recompensas médias recebidas para cada par estado-ação (atividade-recurso) com base no log de eventos:

In [ ]:
avg_costs = (pd.DataFrame(select_res_data.groupby(['Activity','Resource'])['Duration(s)'].mean())
                          .reset_index().round())

time: 15.6 ms (started: 2021-07-04 19:03:39 +00:00)


In [ ]:
R = {s: {a: sys.maxsize for a in resources} for s in states}

def fillR(x):
  R[x['Activity']][x['Resource']] = x['Duration(s)']

_ = avg_costs.apply(fillR, axis = 1)

time: 5.53 ms (started: 2021-07-04 19:04:14 +00:00)


# Algoritmo Iteração de Valor (IV)

## Algoritmo

In [ ]:
# Calcula o Valor - vetor, valor para cada estado
# Retorna o vetor final e a quantidade de iterações executadas 

V_N = {'N':[], 's':[], 'delta': []}
def value_iteration(T, R, V, gama = 0.9, max_iter = 1000, max_error = 0.001):
  for k in range(0, max_iter):
    changed = False
    NEW_V = V.copy()
    for state in V:
      NEW_V[state] = gama * min([R[state][action]+sum([prob * V[next_state] for prob, next_state in T[state][action]]) for action in T[state]])
      delta = abs(NEW_V[state] - V[state])
      V_N['N'].append(k)
      V_N['s'].append(state)
      V_N['delta'].append(delta)
      if delta > max_error:
        changed = True
    if not changed:
      return V, k
    V = NEW_V
    V_total.append(np.array([V[s] for s in V]).sum().round(2))
  return V, k

time: 17.4 ms (started: 2021-07-04 19:04:26 +00:00)


## Aplicação do algoritmo

In [ ]:
init_V = {s: sys.maxsize for s in states}
V_total = []
V, it = value_iteration(T, R, init_V)
V, it

({'W_Assessing the application': 2429.628045559164,
  'W_Calling after sent offers': 2491.129720797761,
  'W_Calling to add missing information to the application': 2932.5823921291712,
  'W_Evaluate fraud': 14189.736926139542,
  'W_Filling in information for the application': 2696.5822516688354,
  'W_Fixing incoming lead': 2399.5501487891006},
 451)

time: 30.9 ms (started: 2021-07-04 19:04:27 +00:00)


## Avaliação de resultado

In [ ]:
d = pd.DataFrame(V_N).groupby('N').mean().reset_index()
d.delta = d.delta/60
px.line(d, x = 'N', y = 'delta', template='plotly_white')

time: 1.46 s (started: 2021-07-04 19:04:27 +00:00)


In [ ]:
px.line(pd.DataFrame(V_total).rename(columns={0:'V_pi'}), y='V_pi', template='plotly_white')

time: 443 ms (started: 2021-07-04 19:04:28 +00:00)


In [ ]:
np.array([V[s] for s in V]).sum().round(2)

27139.21

time: 5.79 ms (started: 2021-07-04 19:04:29 +00:00)


# Algoritmo Iteração de Política

## Algoritmo

In [ ]:
def policy_iteration(P0, T, R, gama = 1.0, max_iter = 100000):
  P = P0.copy()
  V = {s: sys.maxsize for s in states}
  for it in range(0,max_iter):
    changed = False
    NEW_V = V.copy()
    for state in V:
      min = V[state]
      for action in T[state]:
        a_value = R[state][action] + gama * sum([prob * V[next_state] for prob, next_state in T[state][action]])
        if a_value < min:
          changed = True
          min = a_value
          P[state] = action
          NEW_V[state] = a_value
    if not changed:
      return P, it
    V = NEW_V
    V_total.append(np.array([V[s] for s in V]).sum().round(2))
  return P, it

time: 15.9 ms (started: 2021-07-04 19:04:42 +00:00)


## Aplicação do algoritmo

In [ ]:
V_total = []
policy_iteration({s:selected_res[0] for s in states}, T, R)

({'W_Assessing the application': 11179,
  'W_Calling after sent offers': 10863,
  'W_Calling to add missing information to the application': 10863,
  'W_Evaluate fraud': 11304,
  'W_Filling in information for the application': 10863,
  'W_Fixing incoming lead': 10909},
 24970)

time: 1.04 s (started: 2021-07-04 19:04:43 +00:00)


## Avaliação de resultado

In [ ]:
np.array([V[s] for s in V]).sum().round(2)

27139.21

time: 8 ms (started: 2021-07-04 19:04:44 +00:00)


In [ ]:
px.line(pd.DataFrame(V_total).rename(columns={0:'V_pi'}), y='V_pi', template='plotly_white')

time: 458 ms (started: 2021-07-04 19:04:44 +00:00)
